# OpenStreetMap Data Case Study

## Map Area and Source

The map area covers Singapore, SG. I live here.

Map Source: [Mapzen Metro Extract](https://mapzen.com/data/metro-extracts/metro/singapore/)

## Problems Encountered In the Map
A first-pass ETL of `singapore.osm` into a SQLite3 database via `data.py`, several problems were identified and addressed in order:
- *Lorong*, the local equivalent of *street* or *avenue* has been expressed inconsistently - some abbreviated (i.e. `Lor`), some in full (i.e. '`Lorong`')
- *Lorong*s not in consistent order (e.g. some `Toa Payoh Lorong 1`, some `Lorong 1 Geylang`)
- Invalid postcodes (e.g. `#B1-42`, `Johor Bahru`, `<different>`)
- Improperly formed postcodes (e.g. `S 278989`, `Singapore 408564`)
- Non-local 5-digit postcodes (e.g. `18953`)

### Detecting and Fixing Lorongs

The preferred way of referring *Lorong*-style streets is having *Lorong* first, as opposed to regular street names in English (e.g Pickering Street). A number or name follows *Lorong*:
- Lorong 1 Geylang
- Lorong Chuan
- Lorong Ah Soo

Inconsistent abbreviations were found via a regex audit:
```python
def audit_lorong_type(lorong_types, street_name):
    """ Process addr:street tag
    Args:
        lorong_types - Dictionary of keys that are Lorong-street types
        street_name - Street name; 'v' attribute of addr:street
    """
    lorong_type_re = re.compile(r'(\blor\b)|(\blorong\b)', re.IGNORECASE)

    match_result = lorong_type_re.search(street_name)
    if match_result:
        lorong_type = match_result.group()
        lorong_types[lorong_type].add(street_name)
```
and fixed via a simple search/replace function, shown together with the subsequent fix.

__*Lorongs* First!__

If, for example '*Lorong* 1 Geylang' is inverted, i.e 'Geylang *Lorong* 1' - this is still perfectly understandable, however, examples like 'Chuan *Lorong*' is unheard of in local vernacular. Hence, having *Lorong* in front is consistent.

One caveat: if a *Lorong*-style street name begins with a digit, e.g. '2 Lorong Napiri', it would be better practice to ignore it, as it does not refer to a street, so swapping 'Lorong Napiri 2' will not make sense. The code below takes care of the street name cleanup.
```python
def clean_lorong(sname):
    """ Cleans lorong-style street names
    Args:
        sname - Street name; 'v' attribute of addr:street
    Returns:
        Cleaned street name
    """
    verbose = True

    # first find and replace Lor with Lorong
    result = re.sub(r'\bLor\b', 'Lorong', sname)
    # if sname != result: print("original: {}, cleaned: {}".format(sname, result))

    # then swap order of Lorong if applicable
    idx_l = result.find('Lorong')
    # if street name does not begin with lorong & a digit, swap lorong
    if (idx_l > 0) and (result[0] not in set('123456890')):
        newname = result[idx_l:]+ " " + result[:idx_l]
        if verbose: print("swap: {} -> {}".format(result, newname))
        return newname
    else:
        return result
```

### Detecting and Fixing Postcodes
Singapore postal codes are strictly 6 digits in length, with the first 2 digits denoting the postal sector. A simple regular expression can be used to check for non-compliance:
```python
def audit_postcode_type(postcode_types, postcode):
    """ Process addr:postcode tag
    Args:
        postcode_types - Dictionary of keys not compliant to Singapore postcode format
        postcode - Postcode; 'v' attribute of addr:postcode
    """
    postcode_type_re = re.compile(r'^\d{6}$')

    match_result = postcode_type_re.search(postcode)
    if not match_result:
        postcode_types[postcode].add(postcode)
```
The non-compliant postal codes fell into 3 categories - either invalid, malformed or non-Singapore codes. The Mapzen metro extract boundaries extend past Singapore's borders, into both neighbouring Malaysia as well as Indonesia. This accounts for the number of 5-digit postal codes in the data set, which are simply ignored in this analysis.

Malformed codes can be corrected by stripping all other characters and returning the 6-digit postal code.

To resolve the other invalid codes it was possible to either update the postal code to a valid code that does not exist<sup>[1]</sup> (i.e. `000000`), or simply drop the tag. The first option of updating the postal code was chosen, to preserve the tag counts/statistics for later analysis.
```python
def clean_postcode(postcode):
    """ Cleans postcode
    Args:
        postcode - Postcode; 'v' attribute of addr:postcode
    Returns:
        Cleaned postcode
    """
    verbose = True
    pattern = re.compile(r'^\d{5,6}$') # valid codes are 5 or 6-digits

    match_result = pattern.search(postcode)
    if not match_result: # process non-compliant codes
        result = re.sub(r'[^\d]', '', postcode) # clean up malformed codes
    else: # do not process compliant codes
        return postcode

    if len(result) != 6: # if not compliant after clean up, set default postcode
        result = "000000"

    if verbose: print("original: {}, cleaned: {}".format(postcode, result))
    return result
```

## Overview of the Data & Additional Queries
An overview of the data sources is provided in this section. The source OSM XML file was downloaded on **26 Aug 2017** from a standard Mapzen Metro Extract.
### Map Data Quality
The overall data quality with regards to street names is remarkably consistent and clean. This could be due to the fact that Singapore has strict guidelines on how streets are named<sup>[2]</sup>.
### Database and Source Files

File | Size | Type
--- | --- | ---
singapore.osm | 329M | OpenStreetMap XML
singapore.sqlite | 230M | SQLite3 DB
nodes.csv | 119M | Nodes CSV File
nodes_tags.csv | 4.9M | Node Tags CSV File
ways.csv | 14M | Ways CSV File
ways_nodes.csv | 44M | Way Nodes CSV File
ways_tags.csv | 22M | Way Node Tags CSV File

### Nodes & Ways
The number of nodes extracted from `singapore.osm` are as follows:
```sql
sqlite> select count(*) as n from nodes;
n                   
--------------------
1513629             
```
The number of ways extracted from `singapore.osm` are as follows:
```sql
sqlite> select count(*) as n from ways;
n                   
--------------------
237163              
```
### Number of Unique Users
```sql
sqlite> select count(*) from 
            (select u.user from 
                (select user from nodes union all select user from ways) as u 
            group by u.user) as c;
count(*)                 
-------------------------
2144                     

```
### Changesets
A changeset can be open for up to 24 hours, and have maximum of 10,000 entries. The 10-largest changesets are:
```sql
sqlite> select u.changeset, count(*) as n from
   ...> (select changeset from nodes union all select changeset from ways) as u
   ...> group by u.changeset order by n desc limit 10;
changeset             n         
--------------------  ----------
35657677              9875      
29647823              9870      
25268928              9009      
40738823              8938      
15807798              8717      
19508643              8682      
27022028              7786      
24098829              7544      
19934418              7039      
40236725              5949      
```
The size of changesets probably reveals that the users of these changesets are bots.
### Which users made the 10-largest changesets?
```sql
sqlite> select j.user from 
            (select user, changeset from nodes 
            union all 
            select user, changeset from ways) as j 
        join 
            (select u.changeset, count(*) as n from 
                (select changeset from nodes 
                union all 
                select changeset from ways) as u 
             group by u.changeset order by n desc limit 10) as k
        on j.changeset = k.changeset 
        group by j.user;
user                
--------------------
JaLooNz             
mdk                 
```
The top 10 largest changesets were made by only 2 users! This lends more evidence that the these users contributing to the largest changesets are bots. Another check for largest contributors corroborates this as well:
```sql
sqlite> select u.user, count(*) as n from 
            (select user from nodes union all select user from ways) as u 
        group by u.user order by n desc limit 10;
user                  n         
--------------------  ----------
JaLooNz               405158    
berjaya               117460    
rene78                77593     
cboothroyd            72280     
lmum                  50780     
kingrollo             39068     
Luis36995             38823     
ridixcr               38240     
Sihabul Milah         37160     
calfarome             32946     
```

## Suggestions for Improvement
OSM provides GPS longitude and latitude data - but does it also record altitude data? OSM documentation<sup>[3]</sup> on this shows that it could potentially be stored in `ele=` tags. A search on the full Singapore dataset reveals that there are very few such tags compared to the circa 1.5 million nodes in the dataset:
```sql
sqlite> select count(*) from (select * from nodes_tags union all select * from ways_tags) as u where key = 'ele';
54
```
The OSM documentation also reveals the difficulty in recording altitude data:
- Altitude information from consumer-grade GPS devices is often not accurate enough
- Lack of definitive reference altitudes in Singapore (and around the world) for other readings to compare accurately from
- Pressure/barometric-based altimeters are unstable (dependent on temperature and other factors) requiring frequent calibration
- Many reference systems/elevation models of altitude measurement (e.g. Nullebene, Geoid, Referenzellipsoid) requires a significant amount of skill and knowledge from the contributor
- Measuring depth is yet another complication 

A possible solution to this, is to involve big data. While GPS readings from a single contributor may not offer the desired accuracy, GPS tracks from multiple contributors could be taken instead. Building 3-dimensional location heatmaps of a location to achieve decent accuracy. This new data would need to be processed with big data methods because:
- High data volume (fine-grained crowd-sourced data)
- High data veracity (expect noise, bias in the contributor data)
- Moderate data variety (Elevation data could be in the form of GPS, barometric readings, different systems, different formats; raw data could even come from pictures/videos with GPS tags, etc)

Further down the data pipeline, more advanced statistical or machine-learning methods would need to be applied to clean and analyse the data.

As availability and accuracy of the resulting altitude data is dependent on contributor data, the limitations on this approach would be obvious - areas with fewer contributors will have less reliable altitude information. In the future, devices or methods to obtain accurate elevation data may be available and reliance on contributer data would be reduced. 

However, I still think it is a good starting point sufficient for many uses - sufficiently accurate elevation data for hiking, biking, driving.

## Additional Ideas

### Most Common Speed Limit on Asphalt Roads
The official speed limit in Singapore is 50 kilometers per hour on most roads. The query returns 60 instead. Analysing the reasons may require understanding the nature of the roads (e.g. geographical features, location) or source of `maxspeed`.
```sql
sqlite> select key, value, count(*) as n from ways_tags where id in 
            (select id from ways_tags where value = 'asphalt') and key = 'maxspeed' 
        group by key, value order by n desc limit 5;
key                        value       n         
-------------------------  ----------  ----------
maxspeed                   60          3136      
maxspeed                   50          1875      
maxspeed                   70          1336      
maxspeed                   90          668       
maxspeed                   40          224       
```
### Common Cafes
The most common cafe - Starbucks, appears in multiple rows below, due to the different naming conventions of the company. Cleaning data from any `name=` fields will be daunting, as there is considerable variety of correct/valid names with little constraints. Improving the query may require tools that SQL may find challenging.  
```sql
sqlite> select value, count(*) as n from nodes_tags where id in 
            (select id from nodes_tags where value = 'cafe') and key = 'name' 
        group by value order by n desc limit 10;
value                      n         
-------------------------  ----------
Starbucks                  58        
The Coffee Bean & Tea Lea  13        
Toast Box                  8         
Starbucks Coffee           6         
Coffee Bean & Tea Leaf     4         
Coffee Shop                4         
The Coffee Bean and Tea L  4         
Killiney Kopitiam          3         
Koi Cafe                   3         
Lola's Cafe                3         
```
### Unicode Data
Some unicode tags in Chinese language were discovered, which mostly consisted of road names. This raises an interesting point - these unicode data will have to be handled separately in the data audit and cleaning process, because Singapore is a multi-cultural/multi-language society. Inclusions of other language data will increase in future.
```sql
sqlite> select u.value, count(*) as n from 
            (select key, value from nodes_tags union all 
                select key, value from ways_tags) as u 
        where key = 'zh' group by u.value order by n desc limit 10;
value                      n         
-------------------------  ----------
南北大道                       64        
第二通道高速公路                   43        
新柔长堤                       35        
麦士威路                       15        
克罗士街上段                     13        
尼路                         11        
奎因街                        9         
桥南路                        9         
迪沙鲁路                       7         
笨珍路                        6         
```
## Conclusion
This report covered the most pertinent problems encountered when auditing and cleaning the data set and provided a brief overview of the map data including users, nodes and ways as well as some interesting queries.

Street-level data was generally clean but at the same time, scope for further improvements can be made in the areas highlighted from the preceding section.

[1]:https://www.ura.gov.sg/realEstateIIWeb/resources/misc/list_of_postal_districts.htm
[2]:https://www.ura.gov.sg/uol/guidelines/development-control/street-naming
[3]:http://wiki.openstreetmap.org/wiki/Altitude